In [1]:
from langchain.prompts import ChatPromptTemplate
from langchain_ollama import OllamaLLM
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda

model = OllamaLLM(model="gemma3:4b")

In [2]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system","你是一個專業的客服代表，負責回復客戶郵件。請用友善和專業的語調回覆。請不要回覆(公司的資訊和結尾詞)，我會自行手動增加。"),
    ("human","請回覆這封客戶郵件:\n\n{email_content}")
])

In [3]:
def format_email_reply(ai_answer:str) -> str:
    import datetime
    current_time = datetime.datetime.now()
    date_str = current_time.strftime("{y}%Y-%m-%d %H:%M").format(
        y='中華民國,西元'  # 用來替換 {y}
    )
    formatted_reply = f"""{ai_answer}

感謝您的來信，如有其他問題請隨時聯繫我們。

此致
治理客服團隊

回覆時間:{date_str}
"""
    print(f"ai回復的內容:{formatted_reply}")
    return {"formatted_reply":formatted_reply}

format_email = RunnableLambda(format_email_reply)

In [4]:
improve_prompt = ChatPromptTemplate.from_messages(
    [
        ("system","你是郵件品質專家。請評估這封郵件的專業度、友善度、完整度(1-10分)，並且提供具底的改進建議。"),
        ("human","評估並改進這封郵件:\n\n{formatted_reply}")
    ]
)

In [5]:
chain = (prompt_template
        | model
        | StrOutputParser()
        | format_email
        | improve_prompt
        | model
        | StrOutputParser()
        )
        

In [6]:
customer_email = """您好:
我最近購買了櫃公司的產品，但是發現包裝有損壞，而且產品功能與說明書不符。
請問可以退貨或換貨嗎?我已經購買一週了。

期待您的回覆!

王小明"""

In [7]:
result = chain.invoke({
    "email_content":customer_email
})

print(result)

ai回復的內容:您好王小明，

非常感謝您聯繫我們，並詳細說明您遇到的狀況。我們對於您購買的櫃公司產品出現損壞及功能問題深感抱歉，也完全理解您的不滿。

請您放心，我們會盡全力協助您解決這個問題。請您提供更多關於損壞情況的詳細資訊，例如損壞的具體位置、程度，以及產品功能與說明書差異的具體描述，以便我們更快速地處理您的退貨或換貨申請。

我們會盡快與您聯繫，確認這些資訊後，再為您安排下一步處理流程。

再次感謝您的理解與耐心。

感謝您的來信，如有其他問題請隨時聯繫我們。

此致
治理客服團隊

回覆時間:中華民國,西元2025-10-18 15:29

好的，作為郵件品質專家，我來評估並提供改進建議：

**評估結果：**

*   **專業度：** 8/10 - 語句較正式，表達了歉意和承諾，但可以更精煉。
*   **友善度：** 7/10 - 語氣溫和，但缺乏更具體的關懷，例如強調願意為客戶解決問題的決心。
*   **完整度：** 7/10 - 提供了解決問題的步驟，但資訊要求略顯籠統，需要更明確。

**具體改進建議：**

1.  **更精煉開頭：**
    *   原句： “您好王小明，非常感謝您聯繫我們，並詳細說明您遇到的狀況。”
    *   建議： “王小明您好，感謝您聯繫我們，並提供關於櫃公司產品損壞及功能問題的詳細資訊。” (更直接、簡潔)

2.  **強化關懷與同理心：**
    *   原句： “我們對於您購買的櫃公司產品出現損壞及功能問題深感抱歉，也完全理解您的不滿。”
    *   建議： “對於您遇到的困擾，我們深感抱歉。 知道您購買的產品出現了損壞及功能問題，一定會讓您感到非常不便，我們非常理解您的感受。” (更具體地表達對客戶的關心)

3.  **明確資訊要求：**
    *   原句： “請您提供更多關於損壞情況的詳細資訊，例如損壞的具體位置、程度，以及產品功能與說明書差異的具體描述，以便我們更快速地處理您的退貨或換貨申請。”
    *   建議： “為了協助我們更快速地處理您的退貨/換貨申請，請您提供以下資訊：
        *   損壞的具體位置（請提供照片或詳細描述）
        *   損壞的程度（例如：輕微划痕、嚴重凹陷）
        *   產品功能與說明書差異的具體描述（請詳細說明您觀察